In [1]:
import MeCab
import re
import copy
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML
from matplotlib.font_manager import FontProperties


%matplotlib inline


mecab = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')
CABOCHA_ANALYZED_PATH = '../data/ai.ja/ai.ja.txt.cabocha'

In [2]:
INPUT_TEXT_PATH = '../data/ai.ja/ai.ja.txt'

In [3]:
# cabocha をする前に、1文につき1行のフォーマットにしておく
with open(INPUT_TEXT_PATH, "r") as fin:
    texts = []
    for line in fin:
        splitted_lines = line.split("。")
        if len(splitted_lines) == 1:
            texts += splitted_lines
        else:
            for splitted_line in splitted_lines:
                if splitted_line != "\n":
                    texts.append(splitted_line + "。\n")
                else:
                    texts.append(splitted_line)

In [4]:
texts[0:5]

['人工知能\n',
 '\n',
 '人工知能（じんこうちのう、、AI〈エーアイ〉）とは、「『計算（）』という概念と『コンピュータ（）』という道具を用いて『知能』を研究する計算機科学（）の一分野」を指す語。\n',
 '「言語の理解や推論、問題解決などの知的行動を人間に代わってコンピューターに行わせる技術」、または、「計算機（コンピュータ）による知的な情報処理システムの設計や実現に関する研究分野」ともされる。\n',
 '\n']

In [5]:
with open("../data/ai.ja/preprocessed_ai.ja.txt", "w") as fout:
    fout.write("".join(texts))

In [6]:
!cabocha -f1 '../data/ai.ja/preprocessed_ai.ja.txt' >| '../data/ai.ja/ai.ja.txt.cabocha'

# 40. 係り受け解析結果の読み込み（形態素）
# 41. 係り受け解析結果の読み込み（文節・係り受け）

In [7]:
# 40

from typing import List
from collections.abc import Iterable

class Morph:
    def __init__(self, surface, base, pos, pos1):
        self.surface = surface
        self.base = base
        self.pos = pos
        self.pos1 = pos1
        
    def is_sahen_noun(self):
        return (self.pos == "名詞" and self.pos1 == "サ変接続")
    
    def is_wo(self):
        return (self.base == "を" and self.pos1 == "格助詞")
        
    def __str__(self):
        return f"surface[{self.surface}]base[{self.base}]pos[{self.pos}]pos1[{self.pos1}]"
    
class Chunk:
    def __init__(self, morphs, index, dst, srcs):
        self.morphs = morphs
        self.index = index
        self.dst = dst
        if srcs is None:
            self.srcs = []
        else:
            self.srcs = srcs
        if not isinstance(self.srcs, Iterable):
            raise TypeError(f"srcs must be iterable. {self.srcs}")
        
    def __str__(self):
        sentence = ""
        for snt in self.morphs:
            sentence += snt.surface
        return f"{self.index}:{self.dst}:{self.srcs}:{sentence}"
                
    @property
    def surfaces(self):
        sentence = ""
        for snt in self.morphs:
            sentence += snt.surface
        if sentence[-1] in ["。", "、"]:
            sentence = sentence[:-1]
        return sentence
    
    def is_has_noun(self):
        for morph in self.morphs:
            if morph.pos == "名詞":
                return True
                
        return False
    
    def is_has_verb(self):
        for morph in self.morphs:
            if morph.pos == "動詞":
                return True
        return False
    
    def find_verb(self):
        for morph in self.morphs:
            if morph.pos == "動詞":
                return morph.base
        return False
    
    def find_joshi(self):
        for morph in self.morphs:
            if morph.pos == "助詞":
                return morph
        return None
    
    def is_sahen_noun_and_wo(self):
        has_sahen_noun = False
        has_wo = False
        for morph in self.morphs:
            if morph.is_sahen_noun():
                has_sahen_noun = True
            if morph.is_wo():
                has_wo = True
            
        return has_sahen_noun and has_wo
    
class Chunks():
    def __init__(self, chunks: List):
        self.chunks = chunks
        for i, chunk in enumerate(self.chunks):
            if chunk.dst >= len(self.chunks):
                chunk.dst = -1
            if chunk.dst != -1:
                self.chunks[chunk.dst].srcs.append(i)
        
    def __getitem__(self, index):
        return self.chunks[index]
        

# 41
class Sentence():
    def __init__(self, chunks: Chunks):
        self.chunks = chunks
                
    def __str__(self):
        ans = ""
        for i, chunk in enumerate(self.chunks):
            ans += str(chunk) + "\n"
        return ans
    
    def find_kaku_pattern(self):
        verb = None
        joshis = []
        for i, chunk in enumerate(self.chunks):
            if chunk.is_has_verb() == True:
                verb = chunk
                for src_index in chunk.srcs:
                    src_chunk = self.chunks[src_index]
                    morph = src_chunk.find_joshi()
                    if morph is not None:
                        joshis.append((morph.base, src_chunk.surfaces))
                break
                    
        if verb is None:
            return None
        joshis = sorted(joshis, key=lambda x: x[0])
        joshi_chars = [c[0] for c in joshis]
        joshi_chunks = [c[1] for c in joshis]
        joshis_str = " ".join(joshi_chars)
        joshis_chunks_str = " ".join(joshi_chunks)
        ret_str = verb.find_verb()
        if len(joshis_str) > 0:
            ret_str += f" {joshis_str} {joshis_chunks_str}"
                    
        return ret_str
    
    def find_sahen_kaku_pattern(self):
        verb = None
        sahen_noun = None
        joshis = []
        for i, chunk in enumerate(self.chunks):
            if chunk.is_has_verb():
                verb = chunk
                for src_index in verb.srcs:
                    # find nearest sahen wo kaku
                    src_chunk = self.chunks[src_index]
                    morph = src_chunk.find_joshi()
                    if src_chunk.is_sahen_noun_and_wo():
                        sahen_noun = src_chunk
                        
                if verb and sahen_noun:
                    for src_index in verb.srcs:
                        # build joshi list
                        src_chunk = self.chunks[src_index]
                        morph = src_chunk.find_joshi()
                        if morph is not None and not src_chunk == sahen_noun:
                            joshis.append((morph.base, src_chunk.surfaces))
                    break
                else:
                    verb = None
                    sahen_noun = None
                    
        if verb is None:
            return None
        
        # build output string
        joshis = sorted(joshis, key=lambda x: x[0])
        joshi_chars = [c[0] for c in joshis]
        joshi_chunks = [c[1] for c in joshis]
        joshis_str = " ".join(joshi_chars)
        joshis_chunks_str = " ".join(joshi_chunks)
        ret_str = verb.find_verb()
        ret_str = sahen_noun.surfaces + ret_str
        if len(joshis_str) > 0:
            ret_str += f" {joshis_str} {joshis_chunks_str}"
                    
        return ret_str
    
    def build_from_noun_to_root(self):
        patterns = []
        for i, chunk in enumerate(self.chunks):
            if chunk.is_has_noun():
                chunks_to_root = [chunk]
                if chunk.dst == -1:
                    patterns.append(chunk.surfaces)
                    continue
                next_chunk = self.chunks[chunk.dst]
                while True:
                    chunks_to_root.append(next_chunk)
                    if next_chunk.dst == -1:
                        # print(next_chunk.surfaces)
                        break
                    next_chunk = self.chunks[next_chunk.dst]
                        
                # build output
                out_str = chunk.surfaces
                for j, c in enumerate(chunks_to_root):
                    if j == 0:
                        continue
                    out_str += f" -> {c.surfaces}"
                patterns.append(out_str)
                
        return patterns
    
    def search_noun_relation_path(self):
        patterns = []
        for i, chunk in enumerate(self.shunks):
            if chunk.is_has_noun():
                pass
                
                
                
                    

In [8]:

        
def analyze_cabocha_text(text_lines):
    level = 0
    chunks = []
    morphs = []
    for line in text_lines:
        if line == 'EOS\n':
            if len(morphs) != 0:
                chunks.append(Chunk(morphs=morphs, index=clause_num, dst=next_clause, srcs=None))
                morphs = []
                clause_num = None
                next_clause = None
            if len(chunks) != 0:
                yield Sentence(chunks=Chunks(chunks))
                chunks = []
            continue
        else:
            if line[0] == '*':
                if len(morphs) != 0:
                    chunks.append(Chunk(morphs=morphs, index=clause_num, dst=next_clause, srcs=None))
                    # if last character is "。" then it is end of sentence.
                    if morphs[-1].pos1 == "句点":
                        if len(chunks) != 0:
                            yield Sentence(chunks=Chunks(chunks))
                            chunks = []
                    morphs = []
                    clause_num = None
                    next_clause = None
                    
                metas = line.split(' ')
                clause_num = int(metas[1])
                next_clause = int(metas[2].split('D')[0])
                subj_num = int(metas[3].split('/')[0])
                func_num = int(metas[3].split('/')[1])
                score = float(metas[4])
                # print(f"metaline {line}\n{clause_num}:{next_clause}:{subj_num}:{func_num}:{score}")
                continue
                
            cols = line.split('\t')
            res_cols = cols[1].split(',')
            
            morphs.append(Morph(
                cols[0],
                res_cols[6],
                res_cols[0],
                res_cols[1]
            ))
    return
        

def read_lines(path):
    morphs = []
    tmp_text_lines = []
    with open(path) as f:
        for line in f:
            tmp_text_lines.append(line)
     
    return tmp_text_lines

# 42. 係り元と係り先の文節の表示

In [9]:
# 42
for i, sentence in enumerate(analyze_cabocha_text(read_lines(CABOCHA_ANALYZED_PATH)), 1):
    for j, chunk in enumerate(sentence.chunks):
         if chunk.dst != -1:
                print(f"{chunk.surfaces}\t{sentence.chunks[chunk.dst].surfaces}")
    if i == 2:
        break

人工知能	語
（じんこうちのう、	語
AI	〈エーアイ〉）とは
〈エーアイ〉）とは	語
「『計算	（）』という
（）』という	道具を
概念と	道具を
『コンピュータ	（）』という
（）』という	道具を
道具を	用いて
用いて	研究する
『知能』を	研究する
研究する	計算機科学
計算機科学	（）の
（）の	一分野」を
一分野」を	指す
指す	語


# 43. 名詞を含む文節が動詞を含む文節に係るものを抽出

In [10]:
# 43
sentences = []

for i, sentence in enumerate(analyze_cabocha_text(read_lines(CABOCHA_ANALYZED_PATH)), 1):
    sentences.append(sentence)


In [11]:
for i, sentence in enumerate(sentences):
    for j, chunk in enumerate(sentence.chunks):
        if chunk.dst != -1 and chunk.is_has_noun() == True:
            print(f"{chunk.surfaces}\t{sentence.chunks[chunk.dst].surfaces}")
            
    if i == 4:
        break
    

人工知能	語
（じんこうちのう、	語
AI	〈エーアイ〉）とは
〈エーアイ〉）とは	語
「『計算	（）』という
概念と	道具を
『コンピュータ	（）』という
道具を	用いて
『知能』を	研究する
研究する	計算機科学
計算機科学	（）の
一分野」を	指す
「言語の	推論
理解や	推論
推論	問題解決などの
問題解決などの	知的行動を
知的行動を	代わって
人間に	代わって
コンピューターに	行わせる
技術」、または	研究分野」とも
「計算機	（コンピュータ）による
（コンピュータ）による	情報処理システムの
知的な	情報処理システムの
情報処理システムの	実現に関する
設計や	実現に関する
実現に関する	研究分野」とも
研究分野」とも	される
『日本大百科全書(ニッポニカ)』の	解説で
解説で	述べている
情報工学者・通信工学者の	佐藤理史は
佐藤理史は	述べている
次のように	述べている
人間の	知的能力を
知的能力を	実現する
コンピュータ上で	実現する
実現する	技術・ソフトウェア・コンピュータシステム
様々な	技術・ソフトウェア・コンピュータシステム


# 44. 係り受け木の可視化

In [12]:
# 44
import pydot

def create_edge(sentence):
    edge = []
    for j, chunk in enumerate(sentence.chunks):
        if chunk.dst != -1:
            edge.append(((j, chunk.surfaces), (chunk.dst, sentence.chunks[chunk.dst].surfaces)))
            
    return edge

def save_dag_img(edge, name):
    if len(edge) > 0:
        graph = pydot.graph_from_edges(edge, directed=True)
        graph.write_png(name)


In [13]:
create_edge(sentences[1])

[((0, '人工知能'), (17, '語')),
 ((1, '（じんこうちのう、'), (17, '語')),
 ((2, 'AI'), (3, '〈エーアイ〉）とは')),
 ((3, '〈エーアイ〉）とは'), (17, '語')),
 ((4, '「『計算'), (5, '（）』という')),
 ((5, '（）』という'), (9, '道具を')),
 ((6, '概念と'), (9, '道具を')),
 ((7, '『コンピュータ'), (8, '（）』という')),
 ((8, '（）』という'), (9, '道具を')),
 ((9, '道具を'), (10, '用いて')),
 ((10, '用いて'), (12, '研究する')),
 ((11, '『知能』を'), (12, '研究する')),
 ((12, '研究する'), (13, '計算機科学')),
 ((13, '計算機科学'), (14, '（）の')),
 ((14, '（）の'), (15, '一分野」を')),
 ((15, '一分野」を'), (16, '指す')),
 ((16, '指す'), (17, '語'))]

In [14]:
sentence_idx = 2
save_dag_img(create_edge(sentences[sentence_idx]), f"../output/section5/graph_{sentence_idx}.png")

# 45. 動詞の格パターンの抽出
# 46. 動詞の格フレーム情報の抽出

In [15]:
# 45, 46
print(sentences[7].find_kaku_pattern())
kaku_patterns = []
for i, sentence in enumerate(sentences):
    kaku_pattern = sentence.find_kaku_pattern()
    if kaku_pattern is not None:
        kaku_patterns.append(kaku_pattern)
    

with open("../output/section5/kaku_pattern.txt", "w") as fout:
    fout.write("\n".join(kaku_patterns))
    

用いる を 記号処理を


In [16]:
# sort data/kaku_pattern.txt | uniq -c | sort -rn | head
# yuki@MacBook-Pro-2 $ sort data/kaku_pattern.txt | uniq -c | sort -rn | grep 行う
#    5 行う
#    1 行う から が が で に に に に において は は
#    1 行う まで を
#    1 行う と に に は を をめぐって
#    1 行う が て で で に を を
#    1 行う が で を を

# list(analyze_text(["ジョン・マッカーシーはAIに関する最初の会議で人工知能という用語を作り出した。"]))




# 47. 機能動詞構文のマイニング

In [17]:
sahen_noun_kaku_patterns = []
for i, sentence in enumerate(sentences):
    kaku_pattern = sentence.find_sahen_kaku_pattern()
    if kaku_pattern is not None:
        sahen_noun_kaku_patterns.append(kaku_pattern)
    

with open("../output/section5/sahen_norn_kaku_pattern.txt", "w") as fout:
    fout.write("\n".join(sahen_noun_kaku_patterns))
    

In [18]:
sahen_noun_kaku_patterns[:10]

['知的行動を代わる に 人間に',
 '推論・判断をする',
 '記号処理を用いる',
 '注目を集める が 「サポートベクターマシン」が',
 '学習を行う に を 元に 経験を',
 '流行を超える',
 '学習を繰り返す',
 '統計的学習をする で に を を通して ACT-Rでは 元に 推論ルールを 生成規則を通して',
 '進化を見せる て において は 加えて 生成技術において （敵対的生成ネットワーク）は',
 'コンテンツ生成を行う']

# 48. 名詞から根へのパスの抽出

In [19]:
noun_to_roots = []
for i, sentence in enumerate(sentences):
    noun_to_roots += sentence.build_from_noun_to_root()

with open("../output/section5/noun_to_roots.txt", "w") as fout:
    fout.write("\n".join(noun_to_roots))

In [20]:
noun_to_roots[:10]

['人工知能',
 '人工知能 -> 語',
 '（じんこうちのう、 -> 語',
 'AI -> 〈エーアイ〉）とは -> 語',
 '〈エーアイ〉）とは -> 語',
 '「『計算 -> （）』という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す -> 語',
 '概念と -> 道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す -> 語',
 '『コンピュータ -> （）』という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す -> 語',
 '道具を -> 用いて -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す -> 語',
 '『知能』を -> 研究する -> 計算機科学 -> （）の -> 一分野」を -> 指す -> 語']

# 49. 名詞間の係り受けパスの抽出